# Collaborative Filtering

## Load data

In [4]:
import data_processing

#load data
data_processing.load_resources_from_raw_delta_logs(min_num_interactions=6)
data_processing.load_users(min_num_interactions=10)
data_processing.compute_ratings()



FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\A&A\\Desktop\\Anti Confirmation Bias Recommendation System\\data pipeline\\data processing workflow\\data pipeline\\data processing workflow\\data processing session\\resources_info.txt'

In [3]:
path = r'data pipeline\data processing workflow\data processing session\extracted_ratings'
file = open(path)


FileNotFoundError: [Errno 2] No such file or directory: 'data pipeline\\data processing workflow\\data processing session\\extracted_ratings'

## Fit model

In [1]:
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import col 
# from pyspark.ml.evaluation import RegressionEvaluator
# from pyspark.ml.recommendation import ALS
# from pyspark.sql import Row
# import os
# import sys

# # os.environ['PYSPARK_PYTHON'] = sys.executable
# # os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# import data_processing
# import findspark

# findspark.init()



# spark = SparkSession.builder.appName("Collaborative Filtering Flow").getOrCreate()


# path = r'C:\Users\A&A\Downloads\Date Personale Laptop Nagarro\Proiect Licenta\data pipeline\data processing workflow\data processing session\extracted_ratings'


# lines = spark.read.text(path).rdd

# parts = lines.map(lambda row: row.value.split(" "))
# ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),
#                                       rating=float(p[2]) ))


# ratings = spark.createDataFrame(ratingsRDD)
# (training, test) = ratings, ratings
# # # Build the recommendation model using ALS on the training data
# # # Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
# als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",
#            coldStartStrategy="drop")
# model = als.fit(training)

# # # Evaluate the model by computing the RMSE on the test data
# predictions = model.transform(test)
# evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
#                                  predictionCol="prediction")
# rmse = evaluator.evaluate(predictions)
# print("Root-mean-square error = " + str(rmse))

def compute_matrix_factorization():
    from pyspark.sql import SparkSession
    from pyspark.sql.functions import col 
    from pyspark.ml.evaluation import RegressionEvaluator
    from pyspark.ml.recommendation import ALS
    from pyspark.sql import Row
    import os
    import sys

    # os.environ['PYSPARK_PYTHON'] = sys.executable
    # os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

    import data_processing
    import findspark

    findspark.init()

    path = r'data pipeline\data processing workflow\data processing session\extracted_ratings'
    spark = SparkSession.builder.appName("Collaborative Filtering Flow").getOrCreate()


    lines = spark.read.text(path).rdd

    parts = lines.map(lambda row: row.value.split(" "))
    ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), postId=int(p[1]),
                                        rating=float(p[2]) ))


    ratings = spark.createDataFrame(ratingsRDD)
    (training, test) = ratings, ratings
    # # Build the recommendation model using ALS on the training data
    # # Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
    als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="postId", ratingCol="rating",
            coldStartStrategy="drop")
    model = als.fit(training)

    # # Evaluate the model by computing the RMSE on the test data
    predictions = model.transform(test)
    evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                    predictionCol="prediction")
    rmse = evaluator.evaluate(predictions)
    print("Root-mean-square error = " + str(rmse))
    return (ratings,model,rmse)

ratingsPysparkDf,model,rmse = compute_matrix_factorization()

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/c:/Users/A&A/Desktop/Anti Confirmation Bias Recommendation System/data pipeline/data processing workflow/data pipeline/data processing workflow/data processing session/extracted_ratings.

## Extract resources relevant to the latent factors

In [4]:

def extract_baseline_posts(model,cached=True):
    import data_processing
    titles =[
        "Infidelity should not happen when divorce is possible",
         "The default world lingua franca should be Spanish",
         "Buying clothes or goods from factories in the developing world is moral, eve...",
         "The legal owner of a firearm should be responsible for the weapon and anythi...",
         "Airport security screenings do very little to stop deliberate terrorism such as the attacks of 9/11. They are a show put on to make passengers think something is being done.",
         "Luxury watches are useless in the practical sense",
         "The concept of an omniscient (*) and capable creator is not compatible with ...",
         "If whatever makes your character different (sexual identity/disability etc) ...",
         "Cutlery should be placed at the end of a buffet line",
         "If an animal has a big enough population, hunting of it should be allowed"
]
    
    if cached:
        baseline_posts = []
        for title in titles:
            #print(title)
            id = data_processing.res_id[title]
            #print(id)
            baseline_posts.append(data_processing.get_resource_info(id))
        return baseline_posts
    else:    
        '''After advisor approval, these posts shall be hardcoded '''

        from pyspark.sql.functions import expr

        # Extract item factors
        item_factors = model.itemFactors

        # Find the top 5 highest scores for each latent factor
        num_factors = model.rank  # Number of latent factors
        top_scores_per_factor = []

        for i in range(num_factors):
            factor_col = expr(f"features[{i}]")
            top_rows = (item_factors
                        .select("id", factor_col.alias("factor"))
                        .orderBy("factor", ascending=False)
                        .limit(10)
                        .collect())
            top_scores_per_factor.append((i, [(row.id, row.factor) for row in top_rows]))

        #for the final version, the resources will be predefined, to ensure that the reddit posts are not deprecated
        baseline_posts = []
        for factor, top_scores in top_scores_per_factor:
            for rank, (post_id, factor_score) in enumerate(top_scores, start=1):
                if rank ==1:
                    baseline_posts.append({'id':post_id,'score':factor_score,'content':data_processing.id_res[post_id]})
                print(f'title:{data_processing.id_res[post_id]["title"]},url:{data_processing.id_res[post_id]["url"]},factor {factor},score:{factor_score}')
            print("-"*100)
        return baseline_posts

baseline_posts = extract_baseline_posts(model,cached=True)

In [5]:
titles =["Infidelity should not happen when divorce is possible"]
data_processing.get_resource_id("The default world lingua franca should be Spanish")
data_processing.get_resource_id("Buying clothes or goods from factories in the developing world is moral, eve...")
data_processing.get_resource_id("The legal owner of a firearm should be responsible for the weapon and anythi...")
data_processing.get_resource_id("Airport security screenings do very little to stop deliberate terrorism such as the attacks of 9/11. They are a show put on to make passengers think something is being done.")


344

In [6]:
print(baseline_posts[0])

{'title': 'Infidelity should not happen when divorce is possible', 'pos_feedback': ['Nitrousoxide72', 'RedditExplorer89'], 'neg_feedback': ['ripcelinedionhusband', 'Melodic_Echidna', 'joopface', 'dublea', 'ripcelinedionhusband', 'WelfareBear', 'JimboMan1234', '', 'muyamable', 'Nephisimian', 'USoverthem'], 'url': 'https://www.reddit.com//r/changemyview/comments/imh6iw/cmv_infidelity_should_not_happen_when_divorce_is/'}


In [7]:
print(len(baseline_posts),baseline_posts)

10 [{'title': 'Infidelity should not happen when divorce is possible', 'pos_feedback': ['Nitrousoxide72', 'RedditExplorer89'], 'neg_feedback': ['ripcelinedionhusband', 'Melodic_Echidna', 'joopface', 'dublea', 'ripcelinedionhusband', 'WelfareBear', 'JimboMan1234', '', 'muyamable', 'Nephisimian', 'USoverthem'], 'url': 'https://www.reddit.com//r/changemyview/comments/imh6iw/cmv_infidelity_should_not_happen_when_divorce_is/'}, {'title': 'The default world lingua franca should be Spanish', 'pos_feedback': ['NicholasLeo', 'BrotherItsInTheDrum'], 'neg_feedback': ['Igor_Furman', 'muyamable', '', 'MontiBurns', '', 'parentheticalobject'], 'url': 'https://www.reddit.com//r/changemyview/comments/hf49v5/cmv_the_default_world_lingua_franca_should_be/'}, {'title': 'Buying clothes or goods from factories in the developing world is moral, eve...', 'pos_feedback': ['mr-logician', 'thedobya'], 'neg_feedback': ['AnythingApplied', 'AnythingApplied', 'MercurianAspirations', 'StellaAthena'], 'url': 'https://

## Collect data for cold start problem

In [9]:
def init_cold_start(baseline_posts,cached=True):

    import data_processing

    name = input("Please write your name:")
    description = "Dummy description of the task"

    baseline_results =[]

    if cached:
        for res in baseline_posts:
            print(f'Title:{res["title"]}')
            print(f'url: {res["url"]}')
            baseline_score = float(input('How much did this post make you see things trough a new perspective?:'))
            baseline_results.append({'id':data_processing.res_id[res['title']],'user_score':baseline_score})
        return (baseline_results,name)
    else:
        for baseline_post in baseline_posts:
            res = baseline_post["content"]
            print(f'Title:{res["title"]}')
            print(f'url: {res["url"]}')
            baseline_score = float(input('How much did this post make you see things trough a new perspective?:'))
            baseline_results.append({'id':baseline_post['id'],'factor_score':baseline_post['score'],'user_score':baseline_score})
        return (baseline_results,name)


baseline_results,name = init_cold_start(baseline_posts)

Title:Infidelity should not happen when divorce is possible
url: https://www.reddit.com//r/changemyview/comments/imh6iw/cmv_infidelity_should_not_happen_when_divorce_is/
Title:The default world lingua franca should be Spanish
url: https://www.reddit.com//r/changemyview/comments/hf49v5/cmv_the_default_world_lingua_franca_should_be/
Title:Buying clothes or goods from factories in the developing world is moral, eve...
url: https://www.reddit.com//r/changemyview/comments/dwebfc/cmv_buying_clothes_or_goods_from_factories_in_the/
Title:The legal owner of a firearm should be responsible for the weapon and anythi...
url: https://www.reddit.com//r/changemyview/comments/i4f917/cmv_the_legal_owner_of_a_firearm_should_be/
Title:Airport security screenings do very little to stop deliberate terrorism such as the attacks of 9/11. They are a show put on to make passengers think something is being done.
url: https://www.reddit.com/r/changemyview/comments/8a9r3h/cmv_airport_security_screenings_do_very_l

## Fit the model with the new user's data

In [10]:
def add_new_user_ratings(baseline_results):
    ratings_path_file = r'C:\Users\A&A\Downloads\Date Personale Laptop Nagarro\Proiect Licenta\data pipeline\data processing workflow\data processing session\extracted_ratings'

    ratings_file = open(ratings_path_file,'r')


    for line in ratings_file:
        last_line = line
    last_line = last_line.split(" ")
    new_user_id = int(last_line[0]) + 1
    print(new_user_id)
    ratings_file.close()
    ratings_file = open(ratings_path_file,'a')

    for result in baseline_results:
        #ratings_file.write()
        res_id = result['id']
        score = result['user_score']
        ratings_file.write(f'{new_user_id} {res_id} {score}\n')
        #print(f'{new_user_id} {res_id} {score}\n')
    ratings_file.close()
    return new_user_id
user_id = add_new_user_ratings(baseline_results)

294


In [11]:
def compute_matrix_factorization():
    from pyspark.sql import SparkSession
    from pyspark.sql.functions import col 
    from pyspark.ml.evaluation import RegressionEvaluator
    from pyspark.ml.recommendation import ALS
    from pyspark.sql import Row
    import os
    import sys

    # os.environ['PYSPARK_PYTHON'] = sys.executable
    # os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

    import data_processing
    import findspark

    findspark.init()

    path = r'C:\Users\A&A\Downloads\Date Personale Laptop Nagarro\Proiect Licenta\data pipeline\data processing workflow\data processing session\extracted_ratings'

    spark = SparkSession.builder.appName("Collaborative Filtering Flow").getOrCreate()


    lines = spark.read.text(path).rdd

    parts = lines.map(lambda row: row.value.split(" "))
    ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), postId=int(p[1]),
                                        rating=float(p[2]) ))


    ratings = spark.createDataFrame(ratingsRDD)
    (training, test) = ratings, ratings
    # # Build the recommendation model using ALS on the training data
    # # Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
    als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="postId", ratingCol="rating",
            coldStartStrategy="drop")
    model = als.fit(training)

    # # Evaluate the model by computing the RMSE on the test data
    predictions = model.transform(test)
    evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                    predictionCol="prediction")
    rmse = evaluator.evaluate(predictions)
    print("Root-mean-square error = " + str(rmse))
    return (ratings,model,rmse)

ratingsPysparkDf,model,rmse = compute_matrix_factorization()

Root-mean-square error = 0.01893034984413313


## Make recommendations for the new user based on matrix factorization

### Extract ids of already rated posts by user

In [12]:
def extract_posts_already_rated(ratings,user_id = 294):
    from pyspark.sql.functions import col 
    user_ratings = ratings.filter(col("userId") == 294).select("postId").distinct()
    posts_rated_by_user = set()

    for post_id in user_ratings.toPandas().values:
        posts_rated_by_user.add(post_id[0])

    #print(posts_rated_by_user)
    return posts_rated_by_user
current_user_already_rated_posts = extract_posts_already_rated(ratingsPysparkDf)

In [13]:
user_recommendations = model.recommendForAllUsers(3)

In [14]:
#This posts have already been rated by the user

user_recommendations.filter(user_recommendations.userId==294).show(truncate=False)

+------+---------------------------------------------------------+
|userId|recommendations                                          |
+------+---------------------------------------------------------+
|294   |[{1972, 1.3016436}, {1279, 1.1852648}, {2322, 1.0094897}]|
+------+---------------------------------------------------------+



In [15]:
from pyspark.sql.functions import col,lit

def generate_matrix_factorization_recs(ratingsPysparkDf,user_id = 294,number_recs=15,similar=True):
    
    # Generate recommendations for a specific user
    user_ratings = ratingsPysparkDf.filter(col("userId") == user_id).select("postId").distinct()
    #user_ratings.show()
    all_posts = ratingsPysparkDf.select("postId").distinct()
    posts_not_rated_by_user = all_posts.join(user_ratings, on="postId", how="left_anti")

    #posts_not_rated_by_user.show()

    # Recommend top number_recs posts
    posts_not_rated_by_user = posts_not_rated_by_user.withColumn("userId", lit(user_id))
    recommendations = model.transform(posts_not_rated_by_user)
    if similar:
        top_recommendations = recommendations.orderBy(col("prediction").desc()).select("postId", "prediction").limit(number_recs)
    else:
        top_recommendations = recommendations.orderBy(col("prediction")).select("postId", "prediction").limit(number_recs)

    #top_recommendations.show()


    #extract them in a list
    postIds = [p[0] for p in top_recommendations.select("postId").toPandas().values.tolist()]
    predictions = [p[0] for p in top_recommendations.select("prediction").toPandas().values.tolist()]

    #print(postIds)
    #print(predictions)

    matrix_factorization_predictions = []
    for index in range(len(postIds)):
        matrix_factorization_predictions.append({'post_id':postIds[index],'prediction':predictions[index]})
    print(matrix_factorization_predictions)
    return matrix_factorization_predictions

matrix_factorization_predictions = generate_matrix_factorization_recs(ratingsPysparkDf,user_id = 294,similar=True)


[{'post_id': 1972, 'prediction': 1.3016436100006104}, {'post_id': 1279, 'prediction': 1.1852648258209229}, {'post_id': 2816, 'prediction': 0.9350828528404236}, {'post_id': 2118, 'prediction': 0.9242882132530212}, {'post_id': 2801, 'prediction': 0.9189030528068542}, {'post_id': 2382, 'prediction': 0.8939732313156128}, {'post_id': 2016, 'prediction': 0.8786962628364563}, {'post_id': 2070, 'prediction': 0.8668661713600159}, {'post_id': 2546, 'prediction': 0.8338358402252197}, {'post_id': 1420, 'prediction': 0.8120893836021423}, {'post_id': 2925, 'prediction': 0.8095897436141968}, {'post_id': 1826, 'prediction': 0.8074844479560852}, {'post_id': 3661, 'prediction': 0.7994099259376526}, {'post_id': 2561, 'prediction': 0.7991224527359009}, {'post_id': 1288, 'prediction': 0.7989335060119629}]


In [13]:
#after rating
# for rec in matrix_factorization_predictions:
#     current_user_already_rated_posts.add(rec['post_id'])

## Make recommendations for the new user based on similar and opposite users

### Make recommendations based on similar users


### Create similarity matrix of users

In [16]:
def compute_user_similarity_matrix(model):

    from pyspark.sql import SparkSession
    from pyspark.ml.recommendation import ALS
    from pyspark.sql import Row
    import pandas as pd
    from sklearn.metrics.pairwise import cosine_similarity


    # Extract user factors
    user_factors = model.userFactors.orderBy('id')

    # Convert to Pandas DataFrame
    user_factors_pd = user_factors.toPandas()

    # Create a matrix of user factors
    user_ids = user_factors_pd['id'].values
    user_features = pd.DataFrame(user_factors_pd['features'].tolist(), index=user_ids)

    # Compute cosine similarity
    similarity_matrix = cosine_similarity(user_features)

    # Convert the similarity matrix to a DataFrame for easier interpretation
    user_similarity_df = pd.DataFrame(similarity_matrix, index=user_ids, columns=user_ids)

    print(user_similarity_df)
    return user_similarity_df

user_similarity_df = compute_user_similarity_matrix(model)

          2         3         4         5         6         7         8    \
2    1.000000  0.620373 -0.309166 -0.630209 -0.154972  0.243693  0.265376   
3    0.620373  1.000000  0.326082 -0.187345  0.163884  0.077048 -0.433572   
4   -0.309166  0.326082  1.000000  0.513037  0.352590  0.095308 -0.644196   
5   -0.630209 -0.187345  0.513037  1.000000  0.058923  0.029083 -0.683051   
6   -0.154972  0.163884  0.352590  0.058923  1.000000  0.627839 -0.128139   
..        ...       ...       ...       ...       ...       ...       ...   
290 -0.146571 -0.036821  0.363455  0.350617  0.365622  0.175629 -0.019268   
291 -0.015836  0.245979  0.257006 -0.144499  0.422861  0.415727 -0.090254   
292  0.260809  0.345747 -0.004083  0.158294  0.068723  0.237544 -0.489750   
293  0.079526  0.465311  0.438634 -0.198878  0.665281  0.305087 -0.184205   
294 -0.259154 -0.057764  0.108687  0.139631 -0.423141 -0.461920 -0.206150   

          9         10        11   ...       285       286       287  \
2  

In [17]:
#example how to access the similarity between user with itself for  id 2 
user_similarity_df.iloc[2][4]


1.0

In [18]:
user_similarity_df.iloc[292][294]

1.0

### Extract similar and opposite users

In [20]:
def extract_similar_and_opposite_users(user_similarity_df,interogated_user_id=294):
    import numpy as np

    #Note1: if you want to check if user ids are good, 
    #replace [-4:-1] with [-4:] and see if last user has a similarity score of aprox 1

    #Note2: -2 and +2 comes from the fact that we use numpy methods for a pandas dataframe, 
    # dataframe row index starts from 0; column index from 2

    user_similarity_scores_df =user_similarity_df.iloc[interogated_user_id-2]
    similar_users = []

    print(f'Similar users of user with id: {interogated_user_id}')

    for id in np.argsort(user_similarity_scores_df)[-4:-1]:
        user_id = id+2 #colum indexes start from 2 because lowest user_id value is 2
        similarity_score = user_similarity_scores_df[user_id]

        similar_users.append((user_id,similarity_score))
        try:
            print(f'user id: {user_id},similarity score: {similarity_score}')
        except:
            print(f'index error is {user_id} ')
    
    #reverse the list so that most similar user recs come first
    similar_users.reverse()

    print('-'*100)

    opposite_users = []

    print(f'Opposite users of user with id: {interogated_user_id}')
    for id in np.argsort(user_similarity_scores_df)[:3]:
        user_id = id+2
        similarity_score = user_similarity_scores_df[user_id]
        try:
        
            opposite_users.append((user_id,similarity_score))
            print(f'user id: {user_id},similarity score: {similarity_score}')
        except:
            print(f'index error is {user_id} ')
    print('-'*200)
    print(similar_users)
    print(opposite_users)

    return (similar_users,opposite_users)

similar_users,opposite_users = extract_similar_and_opposite_users(user_similarity_df)

Similar users of user with id: 294
user id: 85,similarity score: 0.6239896009545439
user id: 55,similarity score: 0.6569226503842835
user id: 158,similarity score: 0.6771854157354843
----------------------------------------------------------------------------------------------------
Opposite users of user with id: 294
user id: 169,similarity score: -0.8374421864310444
user id: 71,similarity score: -0.7876019883753745
user id: 214,similarity score: -0.7679490775729902
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
[(158, 0.6771854157354843), (55, 0.6569226503842835), (85, 0.6239896009545439)]
[(169, -0.8374421864310444), (71, -0.7876019883753745), (214, -0.7679490775729902)]


### Extract and filter recommendations based on similar and opposite users 

Similar users flow

In [21]:
def generate_sim_recs(ratings_file, baseline_users,current_user_already_rated_posts,num_recs = 30,similar=True):
    num_users = len(baseline_users)
    #print(num_users)
    for index in range(num_users):
        sim_user_id,similarity_score = baseline_users[index]

        print(f'user_id = {sim_user_id}')
        print(f'similarity_score ={similarity_score}')

        sim_user_recs = generate_matrix_factorization_recs(ratings_file,user_id=sim_user_id,number_recs=num_recs,similar=similar)
        current_user_recs = []
        for sim_user_rec_struct in sim_user_recs:

            if sim_user_rec_struct['post_id'] in current_user_already_rated_posts:
                #print('found already rated post')
                continue
            else:
                current_user_rec = {'post_id':sim_user_rec_struct['post_id'],
                                    'prediction':sim_user_rec_struct['prediction'] * similarity_score,
                                    'baseline_user_id':sim_user_id,
                                    'similarity_score': similarity_score,
                                    'baseline_user_prediction':sim_user_rec_struct['prediction'] }

                #print(f'current user recommendation :{current_user_rec}')

                current_user_recs.append(current_user_rec)
    return current_user_recs
similar_users_recs = generate_sim_recs(ratings_file=ratingsPysparkDf,baseline_users=similar_users,current_user_already_rated_posts=current_user_already_rated_posts)

user_id = 158
similarity_score =0.6771854157354843
[{'post_id': 1043, 'prediction': 1.215896725654602}, {'post_id': 344, 'prediction': 1.171457290649414}, {'post_id': 2570, 'prediction': 0.9763970971107483}, {'post_id': 1605, 'prediction': 0.9448734521865845}, {'post_id': 3102, 'prediction': 0.8179556131362915}, {'post_id': 2825, 'prediction': 0.8111118078231812}, {'post_id': 1791, 'prediction': 0.7611003518104553}, {'post_id': 138, 'prediction': 0.7548078894615173}, {'post_id': 2801, 'prediction': 0.7407549619674683}, {'post_id': 1288, 'prediction': 0.7388342618942261}, {'post_id': 2171, 'prediction': 0.7149159908294678}, {'post_id': 2170, 'prediction': 0.7133132219314575}, {'post_id': 1320, 'prediction': 0.6962420344352722}, {'post_id': 2322, 'prediction': 0.6921878457069397}, {'post_id': 1369, 'prediction': 0.6653997302055359}, {'post_id': 1972, 'prediction': 0.6580663919448853}, {'post_id': 427, 'prediction': 0.6328372359275818}, {'post_id': 609, 'prediction': 0.6328372359275818}, 

Opposite users flow

In [22]:
opposite_users_recs = generate_sim_recs(ratings_file=ratingsPysparkDf,baseline_users=opposite_users,current_user_already_rated_posts=current_user_already_rated_posts,similar=False)

user_id = 169
similarity_score =-0.8374421864310444
[{'post_id': 1791, 'prediction': -0.9602436423301697}, {'post_id': 1320, 'prediction': -0.9123225212097168}, {'post_id': 2925, 'prediction': -0.8931249380111694}, {'post_id': 2382, 'prediction': -0.7914252281188965}, {'post_id': 2546, 'prediction': -0.7765008211135864}, {'post_id': 3102, 'prediction': -0.756777286529541}, {'post_id': 2493, 'prediction': -0.7426306009292603}, {'post_id': 1909, 'prediction': -0.7376296520233154}, {'post_id': 2070, 'prediction': -0.7358139157295227}, {'post_id': 1971, 'prediction': -0.7228209376335144}, {'post_id': 2322, 'prediction': -0.7074626684188843}, {'post_id': 1972, 'prediction': -0.6970853805541992}, {'post_id': 3137, 'prediction': -0.6918445825576782}, {'post_id': 1153, 'prediction': -0.6788827776908875}, {'post_id': 673, 'prediction': -0.6612066626548767}, {'post_id': 1555, 'prediction': -0.6554703116416931}, {'post_id': 1279, 'prediction': -0.6437687277793884}, {'post_id': 1420, 'prediction':

Testing on real users logic 

matrix factorization

In [23]:
for rec in matrix_factorization_predictions:
    print(rec)
    break
for rec in similar_users_recs:
    print(rec)
    break
for rec in opposite_users_recs:
    print(rec)
    break

def test_matrix_factorization(recs=matrix_factorization_predictions,tolerance_limit=5):
    import data_processing
    num_rated_posts = 0
    matrix_factorization_results = {}
    for data in recs:
        id = data['post_id']
        res =data_processing.get_resource_info(id)
        print(res['title'])
        print(res['url'])
        is_valid = input('Does the url provide enough info for you to understand the basic ideas expressed in this post?(Y/N):')
        if is_valid.lower() == 'y':
            print("Valid input")
            rec_score = float(input('How much did this recommended post make you see things trough a new perspective?:'))
            matrix_factorization_results[id]={'user_score': rec_score, 'predicted_score':data['prediction'],'validity':True}
            num_rated_posts +=1
        else:
            matrix_factorization_results[id]={'validity':False, 'predicted_score':data['prediction']}

        current_user_already_rated_posts.add(id)
        if num_rated_posts == tolerance_limit:
            return matrix_factorization_results
    return matrix_factorization_results


matrix_factorization_results = test_matrix_factorization(matrix_factorization_predictions) 

{'post_id': 1972, 'prediction': 1.3016436100006104}
{'post_id': 2382, 'prediction': 0.5985916506180975, 'baseline_user_id': 85, 'similarity_score': 0.6239896009545439, 'baseline_user_prediction': 0.9592974781990051}
{'post_id': 1972, 'prediction': 0.7619817456034932, 'baseline_user_id': 214, 'similarity_score': -0.7679490775729902, 'baseline_user_prediction': -0.9922295212745667}
The metric system would be better were it not based on powers of 10
https://www.reddit.com//r/changemyview/comments/imgod1/cmv_the_metric_system_would_be_better_were_it_not/
Valid input
For most people, cast iron skillets aren't worth it
https://www.reddit.com//r/changemyview/comments/dg1s3s/cmv_for_most_people_cast_iron_skillets_arent/
Valid input
Almost all High-school teen drama tv shows would work better if they took pl...
https://www.reddit.com//r/changemyview/comments/sizzw5/cmv_almost_all_highschool_teen_drama_tv_shows/
Valid input
Eugenics is unethical, but not unscientific.
https://www.reddit.com//r/c

In [24]:
len(current_user_already_rated_posts)

15

similar users

In [25]:
import data_processing
def test_user_similarity_recommendations(baseline_users_recs,tolerance_limit =5):
    num_rated_posts = 0
    baseline_user_recs_results = {}
    for data in baseline_users_recs:
        res_id = data['post_id']
        if res_id in current_user_already_rated_posts:
            print('found already recommended post')
            continue
        
        res = data_processing.get_resource_info(res_id)
        baseline_user_id = data['baseline_user_id']
        print(res['title'])
        print(res['url'])
        is_valid = input('Does the url provide enough info for you to understand the basic ideas expressed in this post?(Y/N):')
        if is_valid.lower() == 'y':
            print("Valid input")
            rec_score = float(input('How much did this recommended post make you see things trough a new perspective?:'))
            baseline_user_recs_results[baseline_user_id]={'user_score': rec_score, 'rec_info':data,'validity':True}
            num_rated_posts +=1
        else:
            baseline_user_recs_results[baseline_user_id]={'validity':False, 'rec_info':data}

        current_user_already_rated_posts.add(res_id)
        if num_rated_posts==tolerance_limit:
            return baseline_user_recs_results
    return baseline_user_recs_results


similar_user_recs_results = test_user_similarity_recommendations(similar_users_recs)

Money needs to get out of politics
https://www.reddit.com//r/changemyview/comments/kyl50g/cmv_money_needs_to_get_out_of_politics/
Valid input
found already recommended post
Transexuals are their own thing, not fully male or female.
https://www.reddit.com//r/changemyview/comments/bwrexk/cmv_transexuals_are_their_own_thing_not_fully/
Valid input
Libertarianism is both selfish and poorly thought-out.
https://www.reddit.com//r/changemyview/comments/dj6iqf/cmv_libertarianism_is_both_selfish_and_poorly/
Valid input
found already recommended post
The blame for a mass shooting should be placed on the shooter and not on the...
https://www.reddit.com//r/changemyview/comments/cm1ex1/cmv_the_blame_for_a_mass_shooting_should_be/
Valid input
The loan forgiveness bill might help a little, but doesn't seem to be as "ge...
https://www.reddit.com//r/changemyview/comments/x0t0xw/cmv_the_loan_forgiveness_bill_might_help_a_little/
Definition of fascism is being used incorrectly. Both right and left can be 

In [26]:
len(current_user_already_rated_posts)

21

Opposite users

In [27]:
opposite_user_recs_results = test_user_similarity_recommendations(opposite_users_recs)

found already recommended post
major airlines should just quietly raise their fares by $10 and offer a free...
https://www.reddit.com//r/changemyview/comments/ttp1ee/cmv_major_airlines_should_just_quietly_raise/
Valid input
if we're willing to criticize people like George Washington by today's moral...
https://www.reddit.com//r/changemyview/comments/jj8bim/cmv_if_were_willing_to_criticize_people_like/
Valid input
Guns do not protect against tyranny
https://www.reddit.com//r/changemyview/comments/eybq7d/cmv_guns_do_not_protect_against_tyranny/
Valid input
found already recommended post
Abolishing the police will us unable to protect people from violence
https://www.reddit.com//r/changemyview/comments/gwnw5l/cmv_abolishing_the_police_will_us_unable_to/
Valid input
Having 7x More Liberal Arts/General Studies Majors than STEM is a Serious Pr...
https://www.reddit.com//r/changemyview/comments/nxi37q/cmv_having_7x_more_liberal_artsgeneral_studies/
Valid input


In [28]:
len(current_user_already_rated_posts)

26

saving results


In [29]:
import json

print(opposite_user_recs_results)
print(similar_user_recs_results)
print(matrix_factorization_results)
print(name)




{214: {'user_score': 1.0, 'rec_info': {'post_id': 2561, 'prediction': 0.5977105949073563, 'baseline_user_id': 214, 'similarity_score': -0.7679490775729902, 'baseline_user_prediction': -0.7783206105232239}, 'validity': True}}
{85: {'user_score': 1.0, 'rec_info': {'post_id': 2860, 'prediction': 0.44256873620200476, 'baseline_user_id': 85, 'similarity_score': 0.6239896009545439, 'baseline_user_prediction': 0.7092565894126892}, 'validity': True}}
{1972: {'user_score': 1.0, 'predicted_score': 1.3016436100006104, 'validity': True}, 1279: {'user_score': 1.0, 'predicted_score': 1.1852648258209229, 'validity': True}, 2816: {'user_score': 1.0, 'predicted_score': 0.9350828528404236, 'validity': True}, 2118: {'user_score': 1.0, 'predicted_score': 0.9242882132530212, 'validity': True}, 2801: {'user_score': 1.0, 'predicted_score': 0.9189030528068542, 'validity': True}}
e


In [33]:
import os 

RESULTS_FOLDER_LOCATION = "collaborative_filtering_results"
results_file_location = os.path.join(RESULTS_FOLDER_LOCATION,name+'_collaborative_filtering.json')

results_file = open(results_file_location,'w')

final_user_result={
    'name': name,
    'id':user_id,
    'matrix_factorization':matrix_factorization_results,
    'similar_users':similar_user_recs_results,
    'opposite_users':opposite_user_recs_results
}
json.dump(final_user_result,results_file)


loading results

In [34]:
results_file = open(results_file_location,'r')

print(json.load(results_file))

{'name': 'e', 'id': 294, 'matrix_factorization': {'1972': {'user_score': 1.0, 'predicted_score': 1.3016436100006104, 'validity': True}, '1279': {'user_score': 1.0, 'predicted_score': 1.1852648258209229, 'validity': True}, '2816': {'user_score': 1.0, 'predicted_score': 0.9350828528404236, 'validity': True}, '2118': {'user_score': 1.0, 'predicted_score': 0.9242882132530212, 'validity': True}, '2801': {'user_score': 1.0, 'predicted_score': 0.9189030528068542, 'validity': True}}, 'similar_users': {'85': {'user_score': 1.0, 'rec_info': {'post_id': 2860, 'prediction': 0.44256873620200476, 'baseline_user_id': 85, 'similarity_score': 0.6239896009545439, 'baseline_user_prediction': 0.7092565894126892}, 'validity': True}}, 'opposite_users': {'214': {'user_score': 1.0, 'rec_info': {'post_id': 2561, 'prediction': 0.5977105949073563, 'baseline_user_id': 214, 'similarity_score': -0.7679490775729902, 'baseline_user_prediction': -0.7783206105232239}, 'validity': True}}}


# Final recommendation flow for collaborative filtering